# Intro to GraphX

Wczytać graf Facebooka, krawędzie z pliku musae_facebook_edges.csv, atrybuty page_name oraz page_type z pliku musae_facebook_target.csv.  Policzyć liczbę krawędzi i wierzchołków. (1 p.)

Sprawdzić czy graf jest spójny. Czy dwa podgrafy utworzone dla typów strony politician oraz company też są spójne? (1 p.)

Spośród 1000 stron o najwyższym PageRank znaleźć 50 takich (wypisać page_name i page_type), które mają najmniej połączeń oraz 50 o największej liczbie połączeń. Który typ strony był dominujący w każdej z tych kategorii? Narysować wykres zależności PageRank od liczby krawędzi dla wierzchołków (scatter plot) (2 p.)

Korzystając z Pregel API zaimplementować następujący algorytm. W pierwszym kroku wybrana strona publikuje post fake news. W kolejnym kroku ten post publikowany jest przez ⅓ losowo wybranych kontaktów tej strony. W dalszych krokach, dla każdej strony, która opublikowała już ten post, losowo wybrane ⅓ jej kontaktów publikuje go u siebie. Pokazać jak zmienia się liczba stron które opublikowały post w zależności od liczby kroków. (2 p.)
Opcjonalnie: Sprawdzić powyższą zależność dla współczynnika innego niż 1/3

Narysować wykres rozkładu stopnia wierzchołków w grafie w skali logarytmicznej. Można skorzystać z funkcji obliczającej histogram dla RDD. Czy sieć jest bezskalowa (scale-free)? https://barabasi.com/f/623.pdf  (2 p.)

Useful documentation:

 - https://spark.apache.org/docs/latest/graphx-programming-guide.html
 - https://spark.apache.org/docs/latest/api/java/org/apache/spark/graphx/Pregel.html
 - https://docs.databricks.com/spark/latest/graph-analysis/graph-analysis-graphx-tutorial.html 

Welcome to Scala!

In [1]:
val x:Int = 0

Intitializing Scala interpreter ...

Spark Web UI available at http://25a5e6072dd7:4041
SparkContext available as 'sc' (version = 3.0.0, master = local[*], app id = local-1606501377774)
SparkSession available as 'spark'


x: Int = 0


In [2]:
println("x = " + x)

x = 0


In [3]:
import org.apache.spark.graphx._

import org.apache.spark.graphx._


## Creating graphs
Add some vertices:

In [4]:
val myVertices = sc.makeRDD(Array(
    (1L, "Alice"), 
    (2L, "Bob"),
    (3L, "Charlie"), 
    (4L, "John Doe"), 
    (5L, "Eve")
))

myVertices: org.apache.spark.rdd.RDD[(Long, String)] = ParallelCollectionRDD[0] at makeRDD at <console>:28


Add edges with attributes:

In [5]:
val myEdges = sc.makeRDD(Array(
    Edge(1L, 2L, "is-friends-with"),
    Edge(2L, 3L, "is-friends-with"), 
    Edge(3L, 4L, "is-friends-with"),
    Edge(4L, 5L, "follows"), 
    Edge(3L, 5L, "follows")
))

myEdges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[String]] = ParallelCollectionRDD[1] at makeRDD at <console>:28


Create a graph:

In [6]:
val myGraph = Graph(myVertices, myEdges)

myGraph: org.apache.spark.graphx.Graph[String,String] = org.apache.spark.graphx.impl.GraphImpl@13ca2429


Look at vertices, edges and triplets:

In [7]:
myGraph.vertices.collect

res1: Array[(org.apache.spark.graphx.VertexId, String)] = Array((1,Alice), (2,Bob), (3,Charlie), (4,John Doe), (5,Eve))


In [8]:
myGraph.edges.collect

res2: Array[org.apache.spark.graphx.Edge[String]] = Array(Edge(1,2,is-friends-with), Edge(2,3,is-friends-with), Edge(3,4,is-friends-with), Edge(4,5,follows), Edge(3,5,follows))


In [9]:
myGraph.triplets.collect

res3: Array[org.apache.spark.graphx.EdgeTriplet[String,String]] = Array(((1,Alice),(2,Bob),is-friends-with), ((2,Bob),(3,Charlie),is-friends-with), ((3,Charlie),(4,John Doe),is-friends-with), ((4,John Doe),(5,Eve),follows), ((3,Charlie),(5,Eve),follows))


Iterate over all triplets:

In [10]:
myGraph.triplets.map(
  triplet => triplet.srcAttr + " " + triplet.attr + " " + triplet.dstAttr
).collect.foreach(println(_))

Alice is-friends-with Bob
Bob is-friends-with Charlie
Charlie is-friends-with John Doe
John Doe follows Eve
Charlie follows Eve


## Simple graph analysis
Compute pagerank:

In [11]:
val myPageRankGraph = myGraph.pageRank(0.0001)

myPageRankGraph: org.apache.spark.graphx.Graph[Double,Double] = org.apache.spark.graphx.impl.GraphImpl@208643d1


In [12]:
myPageRankGraph.vertices.collect

res5: Array[(org.apache.spark.graphx.VertexId, Double)] = Array((1,0.4390416708169825), (2,0.8122270910114175), (3,1.1294346981766874), (4,0.9190514175420748), (5,1.7002451224528383))


In [13]:
myGraph.pageRank(0.0001).vertices.join(myVertices).collect.foreach(println)

(1,(0.4390416708169825,Alice))
(2,(0.8122270910114175,Bob))
(3,(1.1294346981766874,Charlie))
(4,(0.9190514175420748,John Doe))
(5,(1.7002451224528383,Eve))


In [14]:
myGraph
  .inDegrees // computes in Degrees
  .foreach(x => println(x._1 + " has " + x._2 + " in degrees."))

5 has 2 in degrees.
4 has 1 in degrees.
3 has 1 in degrees.
2 has 1 in degrees.


## Pregel program example

Goal: collect all the names of friends and followers in all the vertices. 
In every step all vertices send their names to all their connections, and merge together.

Initialize the graph such that all vertices start with empty set:


In [15]:
val initialGraph = myGraph.mapVertices((_,v) => Set(v))

initialGraph: org.apache.spark.graphx.Graph[scala.collection.immutable.Set[String],String] = org.apache.spark.graphx.impl.GraphImpl@77a58762


In [16]:
initialGraph.vertices.collect

res8: Array[(org.apache.spark.graphx.VertexId, scala.collection.immutable.Set[String])] = Array((1,Set(Alice)), (2,Set(Bob)), (3,Set(Charlie)), (4,Set(John Doe)), (5,Set(Eve)))


Run Pregel for 1 step:

In [17]:
val g = Pregel(initialGraph,                                   //  Graph<VD,ED> graph
               Set[String](),                                  //  A initialMsg
               1,                                              //  int maxIterations,
               activeDirection = EdgeDirection.Out)(           //  EdgeDirection activeDirection,
               (id, value, message) => value union message,    //  scala.Function3<Object,VD,A,VD> vprog
               triplet =>                                      //  scala.Function1<EdgeTriplet<VD,ED>,scala.collection.Iterator<scala.Tuple2<Object,A>>> sendMsg,
                    Iterator((triplet.dstId, triplet.srcAttr)),
               (a,b) => a union b                              //  mergeMsg
              )              

g.vertices.collect

g: org.apache.spark.graphx.Graph[scala.collection.immutable.Set[String],String] = org.apache.spark.graphx.impl.GraphImpl@36aed62b
res9: Array[(org.apache.spark.graphx.VertexId, scala.collection.immutable.Set[String])] = Array((1,Set(Alice)), (2,Set(Bob, Alice)), (3,Set(Charlie, Bob)), (4,Set(John Doe, Charlie)), (5,Set(Eve, John Doe, Charlie)))


Run Pregel for 2 steps:

In [18]:
val g = Pregel(initialGraph,                                   //  Graph<VD,ED> graph
               Set[String](),                                  //  A initialMsg
               2,                                              //  int maxIterations,
               activeDirection = EdgeDirection.Out)(           //  EdgeDirection activeDirection,
               (id, value, message) => value union message,    //  scala.Function3<Object,VD,A,VD> vprog
               triplet =>                                      //  scala.Function1<EdgeTriplet<VD,ED>,scala.collection.Iterator<scala.Tuple2<Object,A>>> sendMsg,
                    Iterator((triplet.dstId, triplet.srcAttr)),
               (a,b) => a union b                              //  mergeMsg
              )              

g.vertices.collect

g: org.apache.spark.graphx.Graph[scala.collection.immutable.Set[String],String] = org.apache.spark.graphx.impl.GraphImpl@55b0465e
res10: Array[(org.apache.spark.graphx.VertexId, scala.collection.immutable.Set[String])] = Array((1,Set(Alice)), (2,Set(Bob, Alice)), (3,Set(Charlie, Bob, Alice)), (4,Set(John Doe, Charlie, Bob)), (5,Set(Eve, John Doe, Charlie, Bob)))


Run Pregel for 3 steps:

In [19]:
val g = Pregel(initialGraph,                                   //  Graph<VD,ED> graph
               Set[String](),                                  //  A initialMsg
               3,                                              //  int maxIterations,
               activeDirection = EdgeDirection.Out)(           //  EdgeDirection activeDirection,
               (id, value, message) => value union message,    //  scala.Function3<Object,VD,A,VD> vprog
               triplet =>                                      //  scala.Function1<EdgeTriplet<VD,ED>,scala.collection.Iterator<scala.Tuple2<Object,A>>> sendMsg,
                    Iterator((triplet.dstId, triplet.srcAttr)),
               (a,b) => a union b                              //  mergeMsg
              )              

g.vertices.collect

g: org.apache.spark.graphx.Graph[scala.collection.immutable.Set[String],String] = org.apache.spark.graphx.impl.GraphImpl@65eb27e6
res11: Array[(org.apache.spark.graphx.VertexId, scala.collection.immutable.Set[String])] = Array((1,Set(Alice)), (2,Set(Bob, Alice)), (3,Set(Charlie, Bob, Alice)), (4,Set(John Doe, Charlie, Bob, Alice)), (5,Set(Eve, Bob, Alice, John Doe, Charlie)))


The same but with more explicit type definitions:

In [20]:

// Vertex program - a function to apply to each vertex on a received message
def vprog(id:VertexId, value:Set[String], message:Set[String]): Set[String] = {
    value union message
}

// Send function: 
def sendMsg(et:EdgeTriplet[Set[String],String]): Iterator[Tuple2[VertexId,Set[String]]] = {                 
                    Iterator((et.dstId, et.srcAttr))
}

def mergeMsg(a:Set[String],b:Set[String]): Set[String] = {
    a union b
} 

val g = Pregel(initialGraph,                                 //  Graph<VD,ED> graph
               Set[String](),                                //  A initialMsg
               4,                                            //  int maxIterations,
               activeDirection = EdgeDirection.Out)(         //  EdgeDirection activeDirection,
               vprog,                                        //  scala.Function3<Object,VD,A,VD> vprog
               sendMsg ,                                     //  scala.Function1<EdgeTriplet<VD,ED>,scala.collection.Iterator<scala.Tuple2<Object,A>>> sendMsg,
               mergeMsg                                      //  mergeMsg: (A, A) => A
              )              

g.vertices.collect

vprog: (id: org.apache.spark.graphx.VertexId, value: Set[String], message: Set[String])Set[String]
sendMsg: (et: org.apache.spark.graphx.EdgeTriplet[Set[String],String])Iterator[(org.apache.spark.graphx.VertexId, Set[String])]
mergeMsg: (a: Set[String], b: Set[String])Set[String]
g: org.apache.spark.graphx.Graph[scala.collection.immutable.Set[String],String] = org.apache.spark.graphx.impl.GraphImpl@7d07f9e9
res12: Array[(org.apache.spark.graphx.VertexId, scala.collection.immutable.Set[String])] = Array((1,Set(Alice)), (2,Set(Bob, Alice)), (3,Set(Charlie, Bob, Alice)), (4,Set(John Doe, Charlie, Bob, Alice)), (5,Set(Eve, Bob, Alice, John Doe, Charlie)))
